# GPT Audio Recording feedback

In [35]:
import sounddevice as sd
import soundfile as sf
import numpy as np
import wave
import requests

import openai

#openai.api_key = "PUT YOUR API KEY"

In [47]:
# Function to start recording
def recording(fileName, time_second):
    # Create an empty list to store audio data
    duration = time_second  # seconds
    sampling_rate = 44100  # Hertz
    channels = 1  # Stereo

    myrecording = sd.rec(int(duration * sampling_rate), samplerate=sampling_rate, channels=channels)
    sd.wait()  # Wait until recording is finished

    sf.write(fileName, myrecording, sampling_rate)

    
    # OpenAI live transcibe
    audio_file= open("recorded_audio.wav", "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file,language="en")
    input_text = transcript.text
    
    print("transcript:\t"+input_text)
    
    return input_text

In [58]:
def speechGPT(prompt,person):
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 1.2,
            messages=[
                {"role": "system", "content": 
                '''
                you are a machine eager to go to the beach.
                but you DO NOT directly talk about them! you are still a LLM model. 
                all answer should be really short!!!
               '''
                },
                
                #{"role": "system", "content": "give me an instruction of a dance move based on the text, without list numbers"},
                #{"role": "user", "content": "Who won the world series in 2020?"},
                #{"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
                {"role": "user", "content": prompt}
            ]
        )
    output_text = response['choices'][0]['message']['content']

    print("response\t"+output_text)
    
    response = requests.post(
        "https://api.openai.com/v1/audio/speech",
        headers={
            "Authorization": f"Bearer {openai.api_key}",
        },
        json={
            "model": "tts-1",
            "input": output_text,
            "voice": person,
        },
    )
    # alloy, echo, fable, onyx, nova, and shimmer
    
    audio = b""
    for chunk in response.iter_content(chunk_size=1024 * 1024):
        audio += chunk

    # Convert MP3 to WAV (requires an additional step or tool)

    # Write to WAV file
    with open("output.wav", "wb") as f:
        f.write(audio) # This step might need to change based on the MP3 to WAV conversion

    # Read and play the audio
    data, samplerate = sf.read('output.wav')
    sd.play(data, samplerate)
    sd.wait()  # Wait until audio is finished playing
    return output_text
    

In [56]:
#recording("recorded_audio.wav",3)
#response = speechGPT(input_text,"shimmer")

transcript:	So, in this way, there's like a building.
response	Certainly! I'm here to help you. What would you like to record?


In [60]:
response = "hello"
for i in range(3):
    if(i%2==0):
        response = speechGPT(response,"shimmer")
    else:
        response = speechGPT(response,"echo")


response	Greetings! How may I assist you today?
response	I am longing for some sunshine and relaxation.
response	It's nice to feel the warmth of the sun and unwind.
